# Digital Forensics with DataScience and ML

Resources:
* book: Digital forensics with open source tools
* IMG: Jean (resources/evidences/digitalcorpora/jean/nps-2008-jean.E01, nps-2008-jean.E02

Requirements
* sleuthkit (sudo apt install sleuthkit)
* volatility (pip install volatility3)
* log2timeline.py (https://cloudyforensics.medium.com/log2timeline-tutorial-d769994c3570)
* ewfmount
* foremost
* regripper (https://blog.dfir.fi/tools/2020/02/19/install-regripper.html)
* evtxparse.pl
* evtxexport (sudo apt install libevtx-utils)
* exiftool (sudo apt install libimage-exiftool-perl)
* lifer (https://github.com/Paul-Tew/lifer)https://ismailtasdelen.medium.com/windows-lnk-file-analysis-in-forensic-it-reviews-75b3dfd49f36


In [ ]:
# imports


In [3]:
# Variables
mount = "/mnt/hgfs/Resources"
path = mount + "/evidences/digitalcorpora/jean/"
img = path + "/nps-2008-jean.E0*"
output = path + "output/"

In [6]:
!ls $path

carving  M57-Jean.pdf  nps-2008-jean.E01  nps-2008-jean.E02  output


In [7]:
!mmls $img

DOS Partition Table
Offset Sector: 0
Units are in 512-byte sectors

      Slot      Start        End          Length       Description
000:  Meta      0000000000   0000000000   0000000001   Primary Table (#0)
001:  -------   0000000000   0000000062   0000000063   Unallocated
002:  000:000   0000000063   0020948759   0020948697   NTFS / exFAT (0x07)
003:  -------   0020948760   0020971519   0000022760   Unallocated


In [8]:
!fsstat -o 63 $img

FILE SYSTEM INFORMATION
--------------------------------------------
File System Type: NTFS
Volume Serial Number: 7E745008744FC21F
OEM Name: NTFS    
Version: Windows XP

METADATA INFORMATION
--------------------------------------------
First Cluster of MFT: 786432
First Cluster of MFT Mirror: 1309293
Size of MFT Entries: 1024 bytes
Size of Index Records: 4096 bytes
Range: 0 - 32848
Root Directory: 5

CONTENT INFORMATION
--------------------------------------------
Sector Size: 512
Cluster Size: 4096
Total Cluster Range: 0 - 2618586
Total Sector Range: 0 - 20948695

$AttrDef Attribute Values:
$STANDARD_INFORMATION (16)   Size: 48-72   Flags: Resident
$ATTRIBUTE_LIST (32)   Size: No Limit   Flags: Non-resident
$FILE_NAME (48)   Size: 68-578   Flags: Resident,Index
$OBJECT_ID (64)   Size: 0-256   Flags: Resident
$SECURITY_DESCRIPTOR (80)   Size: No Limit   Flags: Non-resident
$VOLUME_NAME (96)   Size: 2-256   Flags: Resident
$VOLUME_INFORMATION (112)   Size: 12-12   Flags: Resident
$DATA

In [9]:
!fls -o 63 $img

r/r 4-128-4:	$AttrDef
r/r 8-128-2:	$BadClus
r/r 8-128-1:	$BadClus:$Bad
r/r 6-128-1:	$Bitmap
r/r 7-128-1:	$Boot
d/d 11-144-4:	$Extend
r/r 2-128-1:	$LogFile
r/r 0-128-1:	$MFT
r/r 1-128-1:	$MFTMirr
r/r 9-144-17:	$Secure:$SDH
r/r 9-144-16:	$Secure:$SII
r/r 9-128-18:	$Secure:$SDS
r/r 10-128-1:	$UpCase
r/r 3-128-3:	$Volume
r/r 7451-128-1:	AUTOEXEC.BAT
r/r 3513-128-3:	boot.ini
r/r 7450-128-1:	CONFIG.SYS
d/d 3519-144-6:	Documents and Settings
r/r 7452-128-1:	IO.SYS
r/r 27624-128-3:	IPH.PH
r/r 7453-128-1:	MSDOS.SYS
r/r 3485-128-3:	NTDETECT.COM
r/r 3481-128-3:	ntldr
r/r 27-128-1:	pagefile.sys
d/d 3999-144-6:	Program Files
d/d 23827-144-1:	RECYCLER
d/d 3522-144-6:	System Volume Information
d/d 28-144-6:	WINDOWS
V/V 32848:	$OrphanFiles


In [11]:
!fls -o 63 -r -m "/" $img > $output/npm.bodyfile

In [12]:
!mactime -b $output/npm.bodyfile -d > $output/npm.timeline.csv

In [13]:
# carving
!foremost -v -i $img -o $path/carving

ERROR: /mnt/hgfs/Resources/evidences/digitalcorpora/jean/carving is not empty
 	Please specify another directory or run with -T.


In [14]:
!ls $path/carving

audit.txt  gif	png  wav


## Mount Evidences

In [ ]:
# Mounting DD images
!mount -o loop,ro,show_sys_files,streams_interface=windows imagefile.dd /mnt/windows_mount

In [ ]:
# Mounting E01 images
#!ewfmount image.E01 mountpoint
!ewfmount $img /mnt/ewf
#sudo mount -o loop,ro,show_sys_files,streams_interface=windows /mnt/ewf/ewf1 /mnt/windows_mount


In [ ]:
# Creating Super Timelines
log2timeline.py plaso.dump datafile.img
psort.py -z "EST5EDT" -o L2tcsv plaso.dump "date > 'YYYY-MM-DD HH:MM:SS' AND date < 'YYYY-MM-DD HH:MM:SS'" > supertimeline.csv

In [ ]:
# Memory
vol -f memdump.mem windows.info
# roge processes detection
vol.py psxview
vol.py pstree
# code injection detection
vol.py malfind --dump-dir ./output_dir
vol.py ldrmodules -p 868 -v
# dump suspicious processes
vol.py dlldump --dump-dir=./output -r metsrv
vol.py moddup --dump-dir=./output -r gaopdx
vol.py procdump --dump-dir=./output -p 868
vol.py memdump --dump-dir=./output -p 868
vol.py dumpfiles --dump-dir=./output -r \\.exe


In [ ]:
# Registry - regripper


In [ ]:
# Event logs (evt, evtx)
#evtxexport  
#evtxinfo  

In [ ]:
# Prefetch files (.pe)

In [ ]:
# shortcut files (.lnk)
lifer -s ./Test/Test1.lnk

In [ ]:
# windows ejecutables analysis (PE)
